In [61]:
import utils2
sales_util = utils2.SalesUtils('')

import pandas as pd 
import numpy as np
import time
import gc
import pickle

from itertools import product

from collections import Counter
import re
from operator import itemgetter

In [31]:
%%time
input_path = '../data/'
d_parser = lambda x: pd.datetime.strptime(x,'%d.%m.%Y')
df_sales           = pd.read_csv(os.path.join(input_path, 'sales_train.csv'), parse_dates =["date"],date_parser=d_parser)
df_items           = pd.read_csv(os.path.join(input_path, 'items.csv'))
df_item_categories = pd.read_csv(os.path.join(input_path, 'item_categories.csv'))
df_shops           = pd.read_csv(os.path.join(input_path, 'shops.csv'))
df_test            = pd.read_csv(os.path.join(input_path, 'test.csv'))

Wall time: 1min 42s


In [5]:
%%time
############ DELETING OUTLIERS 

# deleting the quantities which are greater than 1001
df_sales = df_sales[df_sales['item_cnt_day']<=1001]
#any item_cnt_day which is less than 0 should be 0
df_sales.loc[df_sales.item_cnt_day < 1,'item_cnt_day'] = 0 

# values more than 55k could be outliers, so deleting all above 55k
df_sales = df_sales[df_sales['item_price']<= 55000]

#item price, should obviously not be less than 0 ... either it should be deleted or imputed.
df_sales = df_sales[df_sales['item_price'] > 0]

############ ADDING DATE ATTRIBUTES

# Adding the date time attributes (like week day, month number, etc.)
df_sales = sales_util.add_date_attributes(df_sales)

############ REPLACING DUPLICATE SHOPS FROM SALES

# based on the above, duplicating as follows
df_sales['shop_id'].replace({0: 57, 1: 58, 11: 10, 40: 39}, inplace=True)

############ REMOVING OUTDATED SHOPS FROM SALES

outdated_shops = [0, 1, 8, 11, 13, 17, 23, 29, 30, 32, 33, 40, 43, 54]
df_sales = df_sales[df_sales['shop_id'].isin(outdated_shops)==False]





Wall time: 0 ns


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [6]:
df_shops.shop_name.unique()

array(['!Якутск Орджоникидзе, 56 фран', '!Якутск ТЦ "Центральный" фран',
       'Адыгея ТЦ "Мега"', 'Балашиха ТРК "Октябрь-Киномир"',
       'Волжский ТЦ "Волга Молл"', 'Вологда ТРЦ "Мармелад"',
       'Воронеж (Плехановская, 13)', 'Воронеж ТРЦ "Максимир"',
       'Воронеж ТРЦ Сити-Парк "Град"', 'Выездная Торговля',
       'Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²',
       'Интернет-магазин ЧС', 'Казань ТЦ "Бехетле"',
       'Казань ТЦ "ПаркХаус" II', 'Калуга ТРЦ "XXI век"',
       'Коломна ТЦ "Рио"', 'Красноярск ТЦ "Взлетка Плаза"',
       'Красноярск ТЦ "Июнь"', 'Курск ТЦ "Пушкинский"',
       'Москва "Распродажа"', 'Москва МТРЦ "Афи Молл"',
       'Москва Магазин С21', 'Москва ТК "Буденовский" (пав.А2)',
       'Москва ТК "Буденовский" (пав.К7)', 'Москва ТРК "Атриум"',
       'Москва ТЦ "Ареал" (Беляево)', 'Москва ТЦ "МЕГА Белая Дача II"',
       'Москва ТЦ "МЕГА Теплый Стан" II',
       'Москва ТЦ "Новый век" (Новокосино)', 'Москва ТЦ "Перловский"',
       'Москва Т

In [39]:
df_shops.groupby(['shop_cat']).count()

,shop_name,shop_id,city
shop_cat,,,
etc,12,12,12
ТК,5,5,5
ТРК,5,5,5
ТРЦ,11,11,11
ТЦ,27,27,27


In [35]:
df_shops['city'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[0])
df_shops['shop_cat'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[1]).astype(str)
df_shops.loc[df_shops.city == '!Якутск', 'city'] = 'Якутск'
shop_category = ['ТК','ТРК','ТРЦ','ТЦ']
df_shops.shop_cat = df_shops.shop_cat.apply(lambda x: x if (x in shop_category) else 'etc')

In [14]:
df_shops['shop_cat'].unique()

array(['Орджоникидзе,', 'ТЦ', 'ТРК', 'ТРЦ', '(Плехановская,', 'Торговля',
       'ул.', 'ЧС', '"Распродажа"', 'МТРЦ', 'Магазин', 'ТК', 'Посад',
       'склад'], dtype=object)

In [38]:
df_shops.groupby(['city']).sum()

,shop_id
city,
Адыгея,2
Балашиха,3
Волжский,4
Вологда,5
Воронеж,21
Выездная,9
Жуковский,21
Интернет-магазин,12
Казань,27


In [21]:
len(df_shops[df_shops.shop_cat=='ТЦ'])

27

In [27]:
df_shops.shop_cat.sum()

array(['etc', 'ТЦ', 'ТРК', 'ТРЦ', 'ул.', 'ТК'], dtype=object)

In [42]:
df_items.item_name.unique()

array(['! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D',
       '!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]',
       '***В ЛУЧАХ СЛАВЫ   (UNV)                    D', ...,
       'Язык запросов 1С:Предприятия 8 (+CD). Хрусталева Е.Ю.',
       'Яйцо для Little Inu', 'Яйцо дракона (Игра престолов)'],
      dtype=object)

In [55]:
df_items['name_1'], df_items['name_2'] = df_items['item_name'].str.split('[', 1).str
df_items['name_1'], df_items['name_3'] = df_items['item_name'].str.split('(', 1).str


df_items['name_2'] = df_items['name_2'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
df_items['name_3'] = df_items['name_3'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
df_items = df_items.fillna('0')

In [108]:
#df1 = Counter(' '.join(df_items['name_2'].values.tolist()).split(' ')).items()
df1 = Counter(' '.join(df_items['name_2'].values).split(' ')).items()
df1 = sorted(df1, key=itemgetter(1))
df1 = pd.DataFrame(df1, columns=['feature', 'count'])
df1.fillna(0)
df1 = df1[(df1['feature'].str.len() > 1) & (df1['count'] > 200)]

df2 = Counter(' '.join(df_items['name_3'].values).split(' ')).items()
df2 = sorted(df2, key=itemgetter(1))
df2 = pd.DataFrame(df2, columns=['feature', 'count'])
df2.fillna(0)
df2 = df2[(df2['feature'].str.len() > 1) & (df2['count'] > 200)]

item_feature_set = pd.concat([df1,df2])
item_feature_set = item_feature_set.drop_duplicates(subset=['feature']).reset_index(drop=True)

def name_correction(x):
    x = x.lower()
    x = x.partition('[')[0]
    x = x.partition('(')[0]
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)
    re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)
    x = x.strip()
    return x

df_items['item_name'] = df_items['item_name'] .apply(lambda x: name_correction(x))
df_items.name_2 = df_items.name_2.apply(lambda x: x[:-1] if x!='0' else '0')

In [109]:
item_feature_set

,feature,count
0,документация,284
1,английская,340
2,русские,399
3,субтитры,400
4,360,465
5,jewel,552
6,xbox,589
7,ps3,611
8,русская,1428
9,цифровая,1995


In [105]:
df1

,feature,count
130,документация,284
131,английская,340
132,русские,399
133,субтитры,400
134,360,465
135,jewel,552
136,xbox,589
137,ps3,611
138,русская,1428
139,цифровая,1995


In [76]:
s = 'this is pakistan and my name is raheel hussain'
#t = Counter(' '.join(s.split()).split(' ')).items()
t = Counter(s.split()).items()

In [70]:
s.split()

['this', 'is', 'pakistan', 'and', 'my', 'name', 'is', 'raheel', 'hussain']

In [81]:
t = Counter(s).items()

In [82]:
t

dict_items([('t', 2), ('h', 3), ('i', 5), ('s', 6), (' ', 8), ('p', 1), ('a', 6), ('k', 1), ('n', 4), ('d', 1), ('m', 2), ('y', 1), ('e', 3), ('r', 1), ('l', 1), ('u', 1)])